# Getting Started

## Loading a model and inspecting it

To begin with, cobrapy comes with bundled models for _Salmonella_ and _E. coli_, as well as a "textbook" model of _E. coli_ core metabolism. To load a test model, type

In [1]:
from __future__ import print_function

import cobra
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("textbook")

The compartment c of metabolite 13dpg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 2pg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 3pg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 6pgc_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 6pgl_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite ac_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment e of metabolite ac_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 

The compartment e of metabolite pi_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite pyr_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment e of metabolite pyr_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite q8_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite q8h2_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite r5p_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite ru5p__D_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite

The reactions, metabolites, genes and compartments attributes of the cobrapy model are a special type of list called a `cobra.DictList`, and each one is made up of `cobra.Reaction`, `cobra.Metabolite`, `cobra.Gene`, `cobra.Compartment` objects respectively.

In [2]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
print(len(model.compartments))

95
72
137
2


When using [Jupyter notebook](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/) this type of information is rendered as a table.

In [3]:
model

<Model e_coli_core at 0x1056a1fd0>

Just like a regular list, objects in the `DictList` can be retrieved by index. For example, to get the 30th reaction in the model (at index 29 because of [0-indexing](https://en.wikipedia.org/wiki/Zero-based_numbering)):

In [4]:
model.reactions[29]

<Reaction EX_glu__L_e at 0x112e5ba90>

Additionally, items can be retrieved by their `id` using the `DictList.get_by_id()` function. For example, to get the cytosolic atp metabolite object (the id is "atp_c"), we can do the following:

In [5]:
model.metabolites.get_by_id("atp_c")

<Metabolite atp_c at 0x112da3048>

As an added bonus, users with an interactive shell such as IPython will be able to tab-complete to list elements inside a list. While this is not recommended behavior for most code because of the possibility for characters like "-" inside ids, this is very useful while in an interactive prompt:

In [6]:
model.reactions.EX_glc__D_e.bounds

(-10.0, 1000.0)

## Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI.

In [7]:
pgi = model.reactions.get_by_id("PGI")
pgi

<Reaction PGI at 0x112ee0160>

We can view the full name and reaction catalyzed as strings

In [8]:
print(pgi.name)
print(pgi.reaction)

glucose-6-phosphate isomerase
g6p_c <=> f6p_c


We can also view reaction upper and lower bounds. Because the `pgi.lower_bound` < 0, and `pgi.upper_bound` > 0, `pgi` is reversible.

In [9]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-1000.0 < pgi < 1000.0
True


We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [10]:
pgi.check_mass_balance()

{}

In order to add a metabolite, we pass in a `dict` with the metabolite object and its coefficient

In [11]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

The reaction is no longer mass balanced

In [12]:
pgi.check_mass_balance()

{'H': -1.0, 'charge': -1.0}

We can remove the metabolite, and the reaction will be balanced once again.

In [13]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


It is also possible to build the reaction from a string. However, care must be taken when doing this to ensure reaction id's match those in the model. The direction of the arrow is also used to update the upper and lower bounds.

In [14]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

unknown metabolite 'green_eggs' created
unknown metabolite 'ham' created


In [15]:
pgi.reaction

'g6p_c --> f6p_c + green_eggs + h_c + ham'

In [16]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

'g6p_c <=> f6p_c'

## Metabolites

We will consider cytosolic atp as our metabolite, which has the id `"atp_c"` in our test model.

In [17]:
atp = model.metabolites.get_by_id("atp_c")
atp

<Metabolite atp_c at 0x112da3048>

We can print out the metabolite name and compartment (cytosol in this case) directly as string.

In [18]:
print(atp.name)
print(atp.compartment)

ATP
c


We can see that ATP is a charged molecule in our model.

In [19]:
atp.charge

-4

We can see the chemical formula for the metabolite as well.

In [20]:
print(atp.formula)

C10H12N5O13P3


The reactions attribute gives a `frozenset` of all reactions using the given metabolite. We can use this to count the number of reactions which use atp.

In [21]:
len(atp.reactions)

13

A metabolite like glucose 6-phosphate will participate in fewer reactions.

In [22]:
model.metabolites.get_by_id("g6p_c").reactions

frozenset({<Reaction PGI at 0x112ee0160>,
           <Reaction GLCpts at 0x112e868d0>,
           <Reaction Biomass_Ecoli_core at 0x112e309e8>,
           <Reaction G6PDH2r at 0x112e7e978>})

## Genes

The `gene_reaction_rule` is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308).

The GPR is stored as the gene_reaction_rule for a Reaction object as a string.

In [23]:
gpr = pgi.gene_reaction_rule
gpr

'b4025'

Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [24]:
pgi.genes

frozenset({<Gene b4025 at 0x112e02748>})

In [25]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

<Gene b4025 at 0x112e02748>

Each gene keeps track of the reactions it catalyzes

In [26]:
pgi_gene.reactions

frozenset({<Reaction PGI at 0x112ee0160>})

Altering the gene_reaction_rule will create new gene objects if necessary and update all relationships.

In [27]:
pgi.gene_reaction_rule = "(spam or eggs)"
pgi.genes

frozenset({<Gene spam at 0x112ee03c8>, <Gene eggs at 0x112ee00f0>})

In [28]:
pgi_gene.reactions

frozenset()

Newly created genes are also added to the model

In [29]:
model.genes.get_by_id("spam")

<Gene spam at 0x112ee03c8>

The `delete_model_genes` function will evaluate the GPR and set the upper and lower bounds to 0 if the reaction is knocked out. This function can preserve existing deletions or reset them using the `cumulative_deletions` flag.

In [30]:
cobra.manipulation.delete_model_genes(
    model, ["spam"], cumulative_deletions=True)
print("after 1 KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))

cobra.manipulation.delete_model_genes(
    model, ["eggs"], cumulative_deletions=True)
print("after 2 KO:  %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))

after 1 KO: -1000 < flux_PGI < 1000
after 2 KO:     0 < flux_PGI <    0


The undelete_model_genes can be used to reset a gene deletion

In [31]:
cobra.manipulation.undelete_model_genes(model)
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

-1000 < pgi < 1000


## Compartments

Like any other component of a cobra.Model, compartments, too, are objects. We will consider the extracellular compartment for now.

In [32]:
model.compartments.e

<Compartment e at 0x112d7a3c8>

It is possible to provide additional information that further describes this compartment in the annotation attribute.

In [33]:
model.compartments.e.annotation = {'sbo': 'SBO:0000247',
'ImaginaryCompDB':'SpecificCompIdentifier'}

In [34]:
model.compartments.e.annotation["sbo"]

'SBO:0000247'

To obtain a list of all the metabolites within a compartment use the function `model.get_metabolites_in_compartments()`.

In [35]:
model.get_metabolites_in_compartment(model.compartments.e)

{<Metabolite h2o_e at 0x112db45f8>,
 <Metabolite etoh_e at 0x112da9a58>,
 <Metabolite h_e at 0x112dc2048>,
 <Metabolite pi_e at 0x112dd57f0>,
 <Metabolite fum_e at 0x112db8080>,
 <Metabolite akg_e at 0x112d9d6a0>,
 <Metabolite ac_e at 0x112d970b8>,
 <Metabolite for_e at 0x112db14a8>,
 <Metabolite succ_e at 0x112de12b0>,
 <Metabolite acald_e at 0x112d97b00>,
 <Metabolite lac__D_e at 0x112dc2710>,
 <Metabolite glu__L_e at 0x112db8f28>,
 <Metabolite nh4_e at 0x112dcc518>,
 <Metabolite co2_e at 0x112da3940>,
 <Metabolite pyr_e at 0x112ddc128>,
 <Metabolite glc__D_e at 0x112db1d68>,
 <Metabolite mal__L_e at 0x112dc2978>,
 <Metabolite o2_e at 0x112dd51d0>,
 <Metabolite gln__L_e at 0x112db8be0>,
 <Metabolite fru_e at 0x112db17f0>}

## Making changes reversibly using models as contexts

Quite often, one wants to make small changes to a model and evaluate the impacts of these. For example, we may want to knock-out all reactions sequentially, and see what the impact of this is on the objective function. One way of doing this would be to create a new copy of the model before each knock-out with `model.copy()`. However, even with small models, this is a very slow approach as models are quite complex objects. Better then would be to do the knock-out, optimizing and then manually resetting the reaction bounds before proceeding with the next reaction. Since this is such a common scenario however, cobrapy allows us to use the model as a context, to have changes reverted automatically.

Please note: Currently, compartment manipulations are not reverted automatically.

In [36]:
model = cobra.test.create_test_model('textbook')
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

The compartment c of metabolite 13dpg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 2pg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 3pg_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 6pgc_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 6pgl_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite ac_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment e of metabolite ac_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite 

The compartment e of metabolite pi_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite pyr_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment e of metabolite pyr_e has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite q8_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite q8h2_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite r5p_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite ru5p__D_c has been replaced with a compartment which already exists in the model the ID of which is identical
The compartment c of metabolite

ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


If we look at those knocked reactions, see that their bounds have all been reverted.

In [37]:
[reaction.bounds for reaction in model.reactions[:5]]

[(-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0)]

Nested contexts are also supported

In [38]:
print('original objective: ', model.objective.expression)
with model:
    model.objective = 'ATPM'
    print('print objective in first context:', model.objective.expression)
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)
print('back to original objective:', model.objective.expression)

original objective:  -1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
print objective in first context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM
back to original objective: -1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core


Most methods that modify the model are supported like this including adding and removing reactions and metabolites and setting the objective. Supported methods and functions mention this in the corresponding documentation.

While it does not have any actual effect, for syntactic convenience it is also possible to refer to the model by a different name than outside the context. Such as

In [39]:
with model as inner:
    inner.reactions.PFK.knock_out